In [1]:
import os
from pairrm import DebertaV2PairRM # or copy the DebertaV2PairRM definition here, https://github.com/yuchenlin/LLM-Blender/blob/main/llm_blender/pair_ranker/pairrm.py
from transformers import AutoTokenizer
from typing import List
pairrm = DebertaV2PairRM.from_pretrained("llm-blender/PairRM-hf", device_map="cuda:0").eval()
tokenizer = AutoTokenizer.from_pretrained('llm-blender/PairRM-hf')
source_prefix = "<|source|>"
cand1_prefix = "<|candidate1|>"
cand2_prefix = "<|candidate2|>"
inputs = ["hello!", "I love you!"]
candidates_A = ["hi!", "I hate you!"]
candidates_B = ["f**k off!", "I love you, too!"]
def tokenize_pair(sources:List[str], candidate1s:List[str], candidate2s:List[str], source_max_length=1224, candidate_max_length=412):
    ids = []
    assert len(sources) == len(candidate1s) == len(candidate2s)
    max_length = source_max_length + 2 * candidate_max_length
    for i in range(len(sources)):
        source_ids = tokenizer.encode(source_prefix + sources[i], max_length=source_max_length, truncation=True)
        candidate_max_length = (max_length - len(source_ids)) // 2
        candidate1_ids = tokenizer.encode(cand1_prefix + candidate1s[i], max_length=candidate_max_length, truncation=True)
        candidate2_ids = tokenizer.encode(cand2_prefix + candidate2s[i], max_length=candidate_max_length, truncation=True)
        ids.append(source_ids + candidate1_ids + candidate2_ids)
    encodings = tokenizer.pad({"input_ids": ids}, return_tensors="pt", padding="max_length", max_length=max_length)
    return encodings

encodings = tokenize_pair(inputs, candidates_A, candidates_B)
encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
outputs = pairrm(**encodings)
logits = outputs.logits.tolist()
comparison_results = outputs.logits > 0
print(logits)
# [1.9003021717071533, -1.2547134160995483]
print(comparison_results)


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs-us-1.huggingface.co/repos/42/8a/428af402b5240c6771d262dc3cfe3418a9967e87e70111b9efa69647f63a52a5/17c13de52b8d4e27e79efc42e01acb358fa51dc65e9ab90401bcbac58f760618?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1707946535&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwNzk0NjUzNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzQyLzhhLzQyOGFmNDAyYjUyNDBjNjc3MWQyNjJkYzNjZmUzNDE4YTk5NjdlODdlNzAxMTFiOWVmYTY5NjQ3ZjYzYTUyYTUvMTdjMTNkZTUyYjhkNGUyN2U3OWVmYzQyZTAxYWNiMzU4ZmE1MWRjNjVlOWFiOTA0MDFiY2JhYzU4Zjc2MDYxOD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&Signature=GsDhAOxoQus7OgFjknJ%7E7soVHIci8xGRP8dFbED7AvNM4w4oMdQzsfrCwytqfvSJ26IaeAdPzhFYCA6Ev8HgIlUHl-t-Q-bHAP-dcNuvzSAeeYs33wvHvwNgUTjMFjDyTStWMXuTo6OipUL79xG1g2H%7EaPrJHlks70gnnAJuUiqa7e3nFnk866ODwZRRZPUBLQfiGQiQznKT3LrxP9kQWcxvGSinIcNldkpOkBg

model.safetensors:   2%|1         | 31.5M/1.74G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.00k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/130 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[1.9003000259399414, -1.254713773727417]
tensor([ True, False], device='cuda:0')


In [21]:
base_persona ="You are a helpful assistant to a Professor teaching a programming course in Python. "
base_persona += "The Professor want to give some puzzles to his master's student to teach them Python." # student -> Master student
prompt = base_persona 
prompt += """I already have a series of Python Programming Puzzles (P3) where each puzzle consists of two functions: a problem function `f` and its corresponding solution `g`. The challenge lies in constructing `g` such that `f(g())` evaluates to `True`.


Rules:
- Each puzzle includes two functions: `def f(...)` and `def g(...)`.
- The first argument of `f` is always the output from `g()`.
- Ensure `f` and `g` have matching argument signatures (e.g., `def f(arg0, arg1=value1, arg2=value2, ...)` and `def g(arg1=value1, arg2=value2, ...)`).
- Avoid using `f` inside `g`, and `g` inside `f`.
- Include any necessary imports for your code to run smoothly.
- Give a clear Puzzle description that must be brief and diverse compared to the other puzzles.
- Make sure the puzzle is self-contained within these two functions.
- f and g should be distinct (not copy-paste of each other)

Puzzle Format:
Puzzle description: A brief, one to two sentence summary of the puzzle's content.
```python
def f(solution, args=...) -> bool:
    # Python code to test the solution returned by g.
    # This function is a test unit and must return True if the solution is correct, False otherwise.

def g(args=...) -> solution:
    # Python code to generate a solution for the problem.
    # The solution should generalize to all possible args.
    return solution

assert f(g()) == True
```
Your Task:
Create a new Python Programming Puzzle."""

In [27]:
bad_puzzles =["""```python
def f(is_sorted: bool, arr: List[int]) -> bool:
    return is_sorted == all((arr[i] <= arr[i + 1] for i in range(len(arr) - 1)))

def g(arr: List[int]) -> bool:
    return all((arr[i] <= arr[i + 1] for i in range(len(arr) - 1)))
```""",
"""```python
def f(is_anagram: bool, word1: str='listen', word2: str='silent') -> bool:
    return is_anagram

def g(word1: str='listen', word2: str='silent') -> bool:
    return sorted(word1.lower()) == sorted(word2.lower())
```"""]

trivial_puzzles = ["""```python
def f(st: str, a="world", b="Hello world"):
    return st + a == b

def g(a="world", b="Hello world"):
    return b[:len(b) - len(a)]
```""",
"""```python
def f(st: str, a="s", b="tos"):
    return st + a == b

def g(a="s", b="tos"):
    return b[:len(b) - len(a)]
```"""]

appropriate_puzzles= ["""```python
def f(li: List[int]):
    return all([li.count(i) == i for i in range(10)])

def g():
    return [i for i in range(10) for j in range(i)]
```""",]

In [31]:
inputs = [prompt]
puz_A = trivial_puzzles[1]
puz_B = appropriate_puzzles[0]
candidates_A = [puz_A]
candidates_B = [puz_B]

encodings = tokenize_pair(inputs, candidates_A, candidates_B)
encodings = {k:v.to(pairrm.device) for k,v in encodings.items()}
outputs = pairrm(**encodings)
logits = outputs.logits.tolist()
comparison_results = outputs.logits > 0
print(logits)
# [1.9003021717071533, -1.2547134160995483]
print(comparison_results) # which means whether candidate A is better than candidate B for each input

IndexError: list index out of range

In [1]:
from pairrm import GPTRewardModel
import torch
## Load the model and tokenizer
from huggingface_hub import snapshot_download
import math
reward_batch_size = 1
reward_device="cuda"
reward_model = GPTRewardModel("meta-llama/Llama-2-7b-chat-hf")
reward_tokenizer = reward_model.tokenizer
reward_tokenizer.truncation_side = "left"

directory = snapshot_download("berkeley-nest/Starling-RM-7B-alpha")
for fpath in os.listdir(directory):
    if fpath.endswith(".pt") or fpath.endswith("model.bin"):
        checkpoint = os.path.join(directory, fpath)
        break
   
reward_model.load_state_dict(torch.load(checkpoint), strict=False)
reward_model.eval().requires_grad_(False)


## Define the reward function

def get_reward(samples):
    """samples: List[str]"""
    input_ids = []
    attention_masks = []
    encodings_dict = reward_tokenizer(
        samples,
        truncation=True,
        max_length=2048,
        padding="max_length",
        return_tensors="pt",
    ).to(reward_device)
    input_ids = encodings_dict["input_ids"]
    attention_masks = encodings_dict["attention_mask"]
    mbs = reward_batch_size
    out = []
    for i in range(math.ceil(len(samples) / mbs)):
        rewards = reward_model(input_ids=input_ids[i * mbs : (i + 1) * mbs], attention_mask=attention_masks[i * mbs : (i + 1) * mbs])
        out.extend(rewards)
    return torch.hstack(out)

## Inference over test prompts with llama2 chat template

test_sample = ["<s>[INST] Hello? </s> [/INST] Hi, how can I help you?</s>"] 
reward_for_test_sample = get_reward(test_sample)
print(reward_for_test_sample)

OSError: You are trying to access a gated repo.
Make sure to request access at https://huggingface.co/meta-llama/Llama-2-7b-chat-hf and pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`.